# [1251. Average Selling Price](https://leetcode.com/problems/average-selling-price/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Prices

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| start_date    | date    |
| end_date      | date    |
| price         | int     |
+---------------+---------+</pre>
(product_id, start_date, end_date) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates the price of the product_id in the period from start_date to end_date.
For each product_id there will be no two overlapping periods. That means there will be no two intersecting periods for the same product_id.
 

Table: UnitsSold

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| purchase_date | date    |
| units         | int     |
+---------------+---------+</pre>
This table may contain duplicate rows.
Each row of this table indicates the date, units, and product_id of each product sold. 
 

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Prices table:
<pre>+------------+------------+------------+--------+
| product_id | start_date | end_date   | price  |
+------------+------------+------------+--------+
| 1          | 2019-02-17 | 2019-02-28 | 5      |
| 1          | 2019-03-01 | 2019-03-22 | 20     |
| 2          | 2019-02-01 | 2019-02-20 | 15     |
| 2          | 2019-02-21 | 2019-03-31 | 30     |
+------------+------------+------------+--------+</pre>
UnitsSold table:
<pre>+------------+---------------+-------+
| product_id | purchase_date | units |
+------------+---------------+-------+
| 1          | 2019-02-25    | 100   |
| 1          | 2019-03-01    | 15    |
| 2          | 2019-02-10    | 200   |
| 2          | 2019-03-22    | 30    |
+------------+---------------+-------+</pre>
Output: 
<pre>+------------+---------------+
| product_id | average_price |
+------------+---------------+
| 1          | 6.96          |
| 2          | 16.96         |
+------------+---------------+</pre>
Explanation: 
Average selling price = Total Price of Product / Number of products sold.
Average selling price for product 1 = ((100 * 5) + (15 * 20)) / 115 = 6.96
Average selling price for product 2 = ((200 * 15) + (30 * 30)) / 230 = 16.96

In [1]:
# pandas schema

import pandas as pd

data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15],
        [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype(
    {'product_id': 'Int64', 'start_date': 'datetime64[ns]', 'end_date': 'datetime64[ns]', 'price': 'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype(
    {'product_id': 'Int64', 'purchase_date': 'datetime64[ns]', 'units': 'Int64'})

In [2]:
#to pyspark df

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

prices_df = spark.createDataFrame(prices)
prices_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/06 18:23:36 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/06 18:23:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 18:23:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+-------------------+-------------------+-----+
|product_id|start_date         |end_date           |price|
+----------+-------------------+-------------------+-----+
|1         |2019-02-17 00:00:00|2019-02-28 00:00:00|5    |
|1         |2019-03-01 00:00:00|2019-03-22 00:00:00|20   |
|2         |2019-02-01 00:00:00|2019-02-20 00:00:00|15   |
|2         |2019-02-21 00:00:00|2019-03-31 00:00:00|30   |
+----------+-------------------+-------------------+-----+


In [3]:
units_sold_df = spark.createDataFrame(units_sold)
units_sold_df.show(truncate=False)

+----------+-------------------+-----+
|product_id|purchase_date      |units|
+----------+-------------------+-----+
|1         |2019-02-25 00:00:00|100  |
|1         |2019-03-01 00:00:00|15   |
|2         |2019-02-10 00:00:00|200  |
|2         |2019-03-22 00:00:00|30   |
+----------+-------------------+-----+


In [4]:
# solving in spark dataframe
import pyspark.sql.functions as F

prices_df.alias('a').join(units_sold_df,
                          on=(prices_df['product_id'] == units_sold_df['product_id'])
                             & ((prices_df['start_date'] <= units_sold_df['purchase_date'])
                                & (prices_df['end_date'] >= units_sold_df['purchase_date'])),
                          how='inner') \
    .groupBy('a.product_id')\
    .agg(F.round(F.sum(F.col('price') * F.col('units')) / F.sum('units'), 2).alias('average_price'))\
    .show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+


In [5]:
# solving in Spark SQL

prices_df.createOrReplaceTempView('prices')
units_sold_df.createOrReplaceTempView('unitsSold')

spark.sql('''
select prices.product_id, round(sum(price*units)/sum(units),2) as average_price 
from prices
inner join unitssold
on prices.product_id = unitssold.product_id
and (prices.start_date<=unitssold.purchase_date and prices.end_date>=unitssold.purchase_date)
group by prices.product_id;
''').show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+


In [6]:
spark.stop()